# Examples

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
from discontinuitypy.core.pipeline import ids_finder
from discontinuitypy.datasets import IDsDataset
from discontinuitypy.utils.basic import resample
import polars as pl
from beforerr.polars import pl_norm

from sunpy.time import TimeRange
from datetime import timedelta
from space_analysis.utils.speasy import Variables

In [ ]:
start = '2019-04-06T12:00'
start = '2019-04-07T01:00'
end = '2019-04-07T12:00'

earth_start = '2019-04-09'
earth_end = '2019-04-14'

# tau = timedelta(seconds=60)
tau = timedelta(seconds=10)
ts = timedelta(seconds=1/264)

In [ ]:
psp_timerange = TimeRange(start, end)
timerange_earth = TimeRange(earth_start, earth_end)

In [ ]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [ ]:
psp_mag_data = Variables(
    dataset="PSP_FLD_L2_MAG_RTN",
    parameters=["psp_fld_l2_mag_RTN"],
    timerange=validate(psp_timerange),
)

In [ ]:
psp_ids_dataset = IDsDataset(
    mag_data=psp_mag_data.to_polars(),
    tau=tau,
    ts=ts,
)

In [ ]:
psp_ids_dataset.find_events()

In [ ]:
psp_ids_dataset.export("../data/psp_ids_dataset.parquet")

## Examples of discontinuities observed by PSP

In [ ]:
#| column: screen
#| layout-ncol: 3
outs = psp_ids_dataset.plot_candidates(plot_fit_data=True)

## Resampled dataset

By visually inspecting data, the most strong discontinuities kind of have time duration about 1-3 seconds. 
So it is reasonable to downsample the data to a lower frequency so that 

- the detection method is more robust, excluding very high frequency wave/noise?
- it is better to apply the derivative method to get the properties of the discontinuities.
- Match `Wind` time resolution.

Or we could fit the $B_l$ to get the properties of the discontinuities.

In [ ]:
every = timedelta(seconds=1/11)
ts = timedelta(seconds=1/11)

data_r = resample(psp_mag_data, every)

In [ ]:
candidates_r = ids_finder(data_r, tau, ts)

In [ ]:
dataset = "PSP_SWP_SPI_SF00_L3_MOM"
parameters = ["DENS", "VEL_RTN_SUN", "TEMP", "SUN_DIST"]

In [ ]:
psp_plasma = Variables(
    dataset=dataset, parameters=parameters, timerange=validate(psp_timerange)
).get_data()

In [ ]:
vec_cols = ["Vx RTN", "Vy RTN", "Vz RTN"]

psp_plasma_r = (
    psp_plasma.to_polars()
    .pipe(resample, every=tau)
    .with_columns(plasma_speed=pl_norm(vec_cols))
    .rename(
        {
            "Density": "plasma_density",
            "Temperature": "plasma_temperature",
        }
    )
)

In [ ]:
psp_ids_dataset = IDsDataset(
    candidates = candidates_r,
    data = data_r,
    plasma_data = psp_plasma_r,
    vec_cols = vec_cols
)

psp_ids_dataset.update_candidates_with_plasma_data()

In [ ]:
def ids_filter(df: pl.DataFrame):
    return df.filter(
        pl.col('index_diff')>1, # interested in strong discontinuities
        pl.col('fit.stat.rsquared')>0.9, # interested in good fits
    )

In [ ]:
#| column: screen
#| layout-ncol: 3
outs = psp_ids_dataset.plot_candidates(add_plasma_params=True, plot_fit_data=True)